# Getting Started

The purpose of the present `Getting Started` section is to give a quick overview of the main objects, methods and functions of the Python `iode` library.
To get a more detailed presentation of all capabilities of `iode`, read the next sections of the tutorial.
    
The [API Reference](../api.rst#api-reference) section of the documentation give you the list of all objects, methods and functions with their individual documentation and examples.

The [Equivalence IODE Report Commands and IODE Python](../equivalence.rst#equivalence-iode-report-commands-and-iode-python) section contains *equivalence tables* between the IODE report syntax and the Python `iode` syntax.

To use the Python `iode` library, the first thing to do is to import objects and functions you need from it:

In [1]:
import numpy as np
from iode import (SAMPLE_DATA_DIR, comments, equations, identities, lists, scalars, 
                  tables, variables, EqMethod)

To know the version of the `iode` library installed on your machine, type:

In [2]:
from iode import __version__
__version__

'7.0.0'

To get the list of objects and functions available in the `iode` li library, use the Python function `dir()`:

In [3]:
import iode
dir(iode)

['AdjustmentMethod',
 'Comments',
 'EQUATION_METHODS_LIST',
 'EQ_TEST_NAMES',
 'ESTIMATION_EPS',
 'ESTIMATION_MAXIT',
 'EqMethod',
 'EqTest',
 'Equation',
 'Equations',
 'FileType',
 'HighToLowType',
 'IODE_DATABASE_TYPE_NAMES',
 'IODE_FILE_TYPES',
 'IODE_FILE_TYPE_NAMES',
 'IODE_LANGUAGES_LIST',
 'Identities',
 'Identity',
 'IodeFileType',
 'IodeType',
 'Lists',
 'LowToHighMethod',
 'LowToHighType',
 'NA',
 'PERIODICITY_LIST',
 'Path',
 'Period',
 'SAMPLE_DATA_DIR',
 'SIMULATION_INITIALIZATION_METHODS',
 'SIMULATION_SORT_ALGORITHMS',
 'Sample',
 'Scalar',
 'Scalars',
 'Simulation',
 'SimulationInitialization',
 'SimulationSort',
 'Table',
 'TableCellAlign',
 'TableCellFont',
 'TableCellType',
 'TableGraphAlign',
 'TableGraphAxis',
 'TableGraphGrid',
 'TableGraphType',
 'TableLang',
 'TableLineType',
 'Tables',
 'Variables',
 'VarsMode',
 'WriteFileExt',
 '__all__',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spe

To print the documentation of an object, method or function in a Python interactive console, use the `help()` function:

In [4]:
# ---- print documentation of a function or method ----
help(equations.load)

Help on method load in module iode.iode_cython:

load(filepath: 'str') method of iode.iode_cython.Equations instance
    load(self, filepath: str)
    
    Load objects stored in file 'filepath' into the current database.
    Erase the database before to load the file.
    
    Parameters
    ----------
    filepath: str
        path to the file to load
    
    Examples
    --------
    >>> from iode import SAMPLE_DATA_DIR
    >>> from iode import comments, variables
    >>> comments.load(f"{SAMPLE_DATA_DIR}/fun.cmt")         # doctest: +ELLIPSIS, +NORMALIZE_WHITESPACE
    Loading .../fun.cmt
    317 objects loaded 
    >>> len(comments)
    317
    
    >>> variables.load(f"{SAMPLE_DATA_DIR}/fun.var")        # doctest: +ELLIPSIS, +NORMALIZE_WHITESPACE
    Loading .../fun.var
    394 objects loaded
    >>> len(variables)
    394



## Load IODE objects

To load IODE objects from a binary file (i.e. with extension `.cmt`, `.eqs`, `.idt`, `.lst`, `.scl`, `.tbl`, `.var`) or from an ASCII file (i.e. with extension `.ac`, `.ae`, `.ai`, `.al`, `.as`, `.at`, `.av`), use the [load()](../_generated/iode.Comments.load.rst#iode.Comments.load) method of the corresponding object. For example:

In [5]:
# ---- load equations, identities, scalars and variables ----
# Note: test binary and ASCII 'fun' files are located in the 'SAMPLE_DATA_DIR' 
#       directory of the 'iode' package
comments.load(f"{SAMPLE_DATA_DIR}/fun.cmt")
equations.load(f"{SAMPLE_DATA_DIR}/fun.eqs")
identities.load(f"{SAMPLE_DATA_DIR}/fun.idt")
lists.load(f"{SAMPLE_DATA_DIR}/fun.lst")
scalars.load(f"{SAMPLE_DATA_DIR}/fun.scl")
tables.load(f"{SAMPLE_DATA_DIR}/fun.tbl")
variables.load(f"{SAMPLE_DATA_DIR}/fun.var")

# ---- print the number of objects present in the above workspaces ----
len(comments), len(equations), len(identities), len(lists), len(scalars), len(tables), len(variables)

Loading C:\soft\Miniconda3\Lib\site-packages\iode\tests\data/fun.cmt
317 objects loaded
Loading C:\soft\Miniconda3\Lib\site-packages\iode\tests\data/fun.eqs
274 objects loaded
Loading C:\soft\Miniconda3\Lib\site-packages\iode\tests\data/fun.idt
48 objects loaded
Loading C:\soft\Miniconda3\Lib\site-packages\iode\tests\data/fun.lst
17 objects loaded
Loading C:\soft\Miniconda3\Lib\site-packages\iode\tests\data/fun.scl
161 objects loaded
Loading C:\soft\Miniconda3\Lib\site-packages\iode\tests\data/fun.tbl
46 objects loaded
Loading C:\soft\Miniconda3\Lib\site-packages\iode\tests\data/fun.var
394 objects loaded


(317, 274, 48, 17, 161, 46, 394)

## Save IODE objects

To save the content of a workspace (or a subset of a workspace), use the [save()](../_generated/iode.Variables.save.rst#iode.Variables.save) method:

In [6]:
# ---- save workspace (or subset) ----
# save the whole workspace
equations.save('equations.eqs')

# save only a subset of the global variables workspace
vars_subset = variables[["ACAF", "ACAG", "AQC", "BQY", "BVY"]]
vars_subset.save('variables_subset.av')    

print("Check content of the variables_subset.av file:\n")
with open("variables_subset.av", "r") as f:
    print(f.read())
print()

Saving c:\usr\Projects\iode-1\doc\source\tutorial\equations.eqs
274 objects saved
Saving c:\usr\Projects\iode-1\doc\source\tutorial\variables_subset.av
Check content of the variables_subset.av file:

sample 1960Y1 2015Y1
ACAF na na na na na na na na na na 1.2130001 5.2020001 9.184 8.0790005 11.332 13.518001 15.784 16.544001 21.489 20.281 21.277 32.417999 24.446999 27.025002 24.504 27.560999 25.542 27.499001 25.353001 17.165001 23.771 26.240999 30.159 34.661999 8.1610022 -13.130997 32.171001 39.935799 29.645657 13.530404919696 10.0466107922005 2.86792273645546 -0.929212509051645 -6.09156498756888 -14.5820944628981 -26.5387895697886 -28.9872879825975 -33.3784257842954 -38.4095177823974 -37.4635096412738 -37.8274288322944 -44.5447926335432 -55.5592898172187 -68.8946543226201 -83.3406251108009 -96.4104198284833 
ACAG na na na na na na na na na na -11.028999 -15.847 -19.288002 -21.814999 -25.447002 -24.618999 -27.770998 -28.839001 -29.434998 -30.411001 -30.353001 -41.060997 -31.178001 -32.6

## Working with workspaces

To get the list of objects names present in a workspace, use the [names](../_generated/iode.Comments.names.rst#iode.Comments.names) attribute of the workspace. 
For example:

In [7]:
# get the list of all IODE lists
lists.names

['COPY',
 'COPY0',
 'COPY1',
 'ENDO',
 'ENDO0',
 'ENDO1',
 'ENVI',
 'IDT',
 'MAINEQ',
 'MYLIST',
 'TOTAL',
 'TOTAL0',
 'TOTAL1',
 'XENVI',
 'XSCENARIO',
 '_SCAL',
 '_SEARCH']

To check if a name is present in a workspace, use the `in` operator. 
For example:

In [8]:
if 'ENVI' in lists:
    print("The 'ENVI' IODE list exists")
else:
    print("'ENVI' IODE list not found")

The 'ENVI' IODE list exists


To iterate over names of a workspace, simply use the Python syntax for the *for loop*:

In [9]:
print("Iterate over all IODE lists names in the Lists workspace:")
for name in lists:
    print(name)

Iterate over all IODE lists names in the Lists workspace:
COPY
COPY0
COPY1
ENDO
ENDO0
ENDO1
ENVI
IDT
MAINEQ
MYLIST
TOTAL
TOTAL0
TOTAL1
XENVI
XSCENARIO
_SCAL
_SEARCH


To get the current used sample for the Variables, use the [sample](../_generated/iode.Variables.sample.rst#iode.Variables.sample) attribute of the [variables](../_generated/iode.Variables.rst#iode.Variables) workspace:

In [10]:
# current used sample
variables.sample

Sample("1960Y1:2015Y1")

### Get - add - update - delete IODE object(s)

In a similar way to Python dictionaries, you can [get](../_generated/iode.Equations.__getitem__.rst), [add](../_generated/iode.Equations.__setitem__.rst), [update](../_generated/iode.Equations.__setitem__.rst) and [delete](../_generated/iode.Equations.__delitem__.rst) IODE objects in a workspace using the `[]` operator.

* To extract an IODE object from a workspace, use the syntax: `my_obj = workspace[name]`.
* To add an IODE object to a workspace, use the syntax: `workspace[new_name] = new_obj`.
* To update an IODE object in a workspace, use the syntax: `workspace[name] = new_value`.
* To delete an IODE object from a workspace, use the syntax: `del workspace[name]`.

It is also possible to add or update multiple IODE objects at once by passing the list of names to the `[]` operator and the list of new/updated values after the `=` operator: `workspace[list_of_names] = list_of_new_values`.
To delete multiple IODE objects at once, pass the list of names to delete to the `[]` operator: `del workspace[list_of_names]`.

### Workspace subsets

IODE workspaces can contains a lot objects and it can be sometimes easier to work on a subset of the objects present in a workspace. To get a subset of an IODE workspace, a *pattern* can be passed to the `[]` operator. 
A (sub-)`pattern` is a list of characters representing a group of object names. It includes some special characters which have a special meaning:

* `*` : any character sequence, even empty
* `?` : any character (one and only one)
* `@` : any alphanumerical char [A-Za-z0-9]
* `&` : any non alphanumerical char
* `|` : any alphanumeric character or none at the beginning and end of a string 
* `!` : any non-alphanumeric character or none at the beginning and end of a string 
* `\` : escape the next character

The *pattern* can contain sub-patterns, as well as, object names. The sub-patterns and object names are separated by a *separator* character which is either:

* a whitespace `' '`
* a comma `,`
* a semi-colon `;`
* a tabulation `\t`
* a newline `\n`

Note that the *pattern* can contain references to IODE lists which are prefixed with the symbol `$`.

<div class="alert alert-info">

**Note**

When an IODE object is added, updated or deleted from a subset of a workspace, the change is also applied to the global workspace. For example, if an equation is added to the subset of the *equations* workspace, the the change is also applied to the global *equations* database.

To create an isolate subset of a workspace, use the [copy](../_generated/iode.Equations.copy.rst#iode.Equations.copy) method. This method returns a new workspace in which each object is a copy of the original object from the global workspace. Any change made to the *copied subset* will not be applied to the global workspace. This can be useful for example if you want to save previous values of scalars before estimating an equation or a block of equations and then restore the original values if the estimated values are not satisfying.

</div>

The examples below show how to update, add and delete objects for each IODE workspace type.

### Get - add - update - delete comment(s)

Add one comment:

In [11]:
comments["NEW"] = "A new comment"
comments["NEW"]

'A new comment'

Update a comment:

In [12]:
comments["NEW"] = "New Value"
comments["NEW"]

'New Value'

Delete a comment:

In [13]:
comments.get_names("A*")

['ACAF', 'ACAG', 'AOUC', 'AQC']

In [14]:
del comments["ACAF"]
comments.get_names("A*")

['ACAG', 'AOUC', 'AQC']

Working on a subset:

In [15]:
# 1) get subset
comments_subset = comments["A*"]
comments_subset.names

['ACAG', 'AOUC', 'AQC']

In [16]:
# 2) add a comment to the subset
comments_subset["A0"] = "New Comment"
comments_subset["A0"]

'New Comment'

In [17]:
# --> new comment also appears in the global workspace
"A0" in comments, comments["A0"]

(True, 'New Comment')

In [18]:
# 3) update a comment in the subset
comments_subset["A0"] = "Updated Comment"
comments_subset["A0"]

'Updated Comment'

In [19]:
# --> comment is also updated in the global workspace
comments["A0"]

'Updated Comment'

In [20]:
# delete comment from the subset
del comments_subset["A0"]
comments_subset.names

['ACAG', 'AOUC', 'AQC']

In [21]:
# NOTE: the comment has also been deleted from the global database
"A0" in comments

False

In [22]:
comments.get_names("A*")

['ACAG', 'AOUC', 'AQC']

### Get - add - update - delete equation(s)

Add one equation:

In [23]:
equations["TEST"] = "TEST := 0"
equations["TEST"]

Equation(endogenous = 'TEST',
         lec = 'TEST := 0',
         method = 'LSQ',
         from_period = '1960Y1',
         to_period = '2015Y1')

Update an equation:

In [24]:
equations["ACAF"]

Equation(endogenous = 'ACAF',
         lec = '(ACAF/VAF[-1]) :=acaf1+acaf2*GOSF[-1]+\nacaf4*(TIME=1995)',
         method = 'LSQ',
         from_period = '1980Y1',
         to_period = '1996Y1',
         block = 'ACAF',
         tests = {corr = 1,
                  dw = 2.32935,
                  fstat = 32.2732,
                  loglik = 83.8075,
                  meany = 0.00818467,
                  r2 = 0.821761,
                  r2adj = 0.796299,
                  ssres = 5.19945e-05,
                  stderr = 0.00192715,
                  stderrp = 23.5458,
                  stdev = 0.0042699},
         date = '12-06-1998')

In [25]:
# update only the LEC
equations["ACAF"] = "(ACAF/VAF[-1]) := acaf1 + acaf2 * GOSF[-1] + acaf4 * (TIME=1995)"
equations["ACAF"]

Equation(endogenous = 'ACAF',
         lec = '(ACAF/VAF[-1]) := acaf1 + acaf2 * GOSF[-1] + acaf4 * (TIME=1995)',
         method = 'LSQ',
         from_period = '1980Y1',
         to_period = '1996Y1',
         block = 'ACAF',
         tests = {corr = 1,
                  dw = 2.32935,
                  fstat = 32.2732,
                  loglik = 83.8075,
                  meany = 0.00818467,
                  r2 = 0.821761,
                  r2adj = 0.796299,
                  ssres = 5.19945e-05,
                  stderr = 0.00192715,
                  stderrp = 23.5458,
                  stdev = 0.0042699},
         date = '12-06-1998')

In [26]:
# update block and sample of a block of equations to estimation (dictionary)
estim_sample = "2000Y1:2010Y1"
block = "ACAF; ACAG; AOUC"
for eq_name in block.split(';'):
    equations[eq_name] = {"sample": estim_sample, "block": block}

(equations["ACAF"].sample, equations["ACAG"].sample, equations["AOUC"].sample)

(Sample("2000Y1:2010Y1"), Sample("2000Y1:2010Y1"), Sample("2000Y1:2010Y1"))

In [27]:
(equations["ACAF"].block, equations["ACAG"].block, equations["AOUC"].block)

('ACAF; ACAG; AOUC', 'ACAF; ACAG; AOUC', 'ACAF; ACAG; AOUC')

In [28]:
# update lec, method, sample and block
equations["ACAF"].lec = "(ACAF/VAF[-1]) := acaf2 * GOSF[-1] + acaf4 * (TIME=1995)"
equations["ACAF"].method = EqMethod.MAX_LIKELIHOOD
# new equation sample is from 1990Y1 to the last year of Variables
equations["ACAF"].sample = "1990Y1:"
equations["ACAF"].block = "ACAF"
equations["ACAF"]

Equation(endogenous = 'ACAF',
         lec = '(ACAF/VAF[-1]) := acaf2 * GOSF[-1] + acaf4 * (TIME=1995)',
         method = 'MAX_LIKELIHOOD',
         from_period = '1990Y1',
         to_period = '2015Y1',
         block = 'ACAF',
         tests = {corr = 1,
                  dw = 2.32935,
                  fstat = 32.2732,
                  loglik = 83.8075,
                  meany = 0.00818467,
                  r2 = 0.821761,
                  r2adj = 0.796299,
                  ssres = 5.19945e-05,
                  stderr = 0.00192715,
                  stderrp = 23.5458,
                  stdev = 0.0042699},
         date = '12-06-1998')

Delete an equation:

In [29]:
equations.get_names("A*")

['ACAF', 'ACAG', 'AOUC']

In [30]:
del equations["ACAF"]
equations.get_names("A*")

['ACAG', 'AOUC']

Working on a subset:

In [31]:
# 1) get subset
equations_subset = equations["A*"]
equations_subset.names

['ACAG', 'AOUC']

In [32]:
# 2) add a equation to the subset
equations_subset["AOUC_"] = "AOUC_ := ((WCRH/QL)/(WCRH/QL)[1990Y1]) * (VAFF/(VM+VAFF))[-1] + PM * (VM/(VAFF+VM))[-1]"
equations_subset["AOUC_"]

Equation(endogenous = 'AOUC_',
         lec = 'AOUC_ := ((WCRH/QL)/(WCRH/QL)[1990Y1]) * (VAFF/(VM+VAFF))[-1] + PM * (VM/(VAFF+VM))[-1]',
         method = 'LSQ',
         from_period = '1960Y1',
         to_period = '2015Y1')

In [33]:
# --> new equation also appears in the global workspace
"AOUC_" in equations

True

In [34]:
equations["AOUC_"]

Equation(endogenous = 'AOUC_',
         lec = 'AOUC_ := ((WCRH/QL)/(WCRH/QL)[1990Y1]) * (VAFF/(VM+VAFF))[-1] + PM * (VM/(VAFF+VM))[-1]',
         method = 'LSQ',
         from_period = '1960Y1',
         to_period = '2015Y1')

In [35]:
# 3) update a equation in the subset
equations_subset["AOUC_"] = "AOUC_ := ((WCRH/QL)/(WCRH/QL)[1990Y1]) * (VAFF/(VM+VAFF))[-1]"
equations_subset["AOUC_"]           

Equation(endogenous = 'AOUC_',
         lec = 'AOUC_ := ((WCRH/QL)/(WCRH/QL)[1990Y1]) * (VAFF/(VM+VAFF))[-1]',
         method = 'LSQ',
         from_period = '1960Y1',
         to_period = '2015Y1')

In [36]:
# --> equation is also updated in the global workspace
equations["AOUC_"]

Equation(endogenous = 'AOUC_',
         lec = 'AOUC_ := ((WCRH/QL)/(WCRH/QL)[1990Y1]) * (VAFF/(VM+VAFF))[-1]',
         method = 'LSQ',
         from_period = '1960Y1',
         to_period = '2015Y1')

In [37]:
# delete one equation from a subset of the global database
equations_subset = equations["D*"]
equations_subset.names

['DEBT',
 'DPU',
 'DPUF',
 'DPUG',
 'DPUGO',
 'DPUH',
 'DPUU',
 'DTF',
 'DTH',
 'DTH1',
 'DTH1C']

In [38]:
del equations_subset["DPUGO"]
equations_subset.names

['DEBT', 'DPU', 'DPUF', 'DPUG', 'DPUH', 'DPUU', 'DTF', 'DTH', 'DTH1', 'DTH1C']

In [39]:
# NOTE: the equation has also been deleted from the global database
equations.get_names("D*")

['DEBT', 'DPU', 'DPUF', 'DPUG', 'DPUH', 'DPUU', 'DTF', 'DTH', 'DTH1', 'DTH1C']

### Get - add - update - delete identity(ies)

Add one identity:

In [40]:
identities["BDY"] = "YN - YK"
identities["BDY"]

Identity('YN - YK')

Update an identity:

In [41]:
identities["AOUC"]

Identity('((WCRH/QL)/(WCRH/QL)[1990Y1])*(VAFF/(VM+VAFF))[-1]+PM*(VM/(VM+VAFF))[-1]')

In [42]:
identities["AOUC"] = '(WCRH / WCRH[1990Y1]) * (VAFF / (VM+VAFF))[-1] + PM * (VM / (VM+VAFF))[-1]'
identities["AOUC"]

Identity('(WCRH / WCRH[1990Y1]) * (VAFF / (VM+VAFF))[-1] + PM * (VM / (VM+VAFF))[-1]')

Delete an identity:

In [43]:
identities.get_names("W*")

['W', 'WBGR', 'WCRH', 'WMINR', 'WO']

In [44]:
del identities["W"]
identities.get_names("W*")

['WBGR', 'WCRH', 'WMINR', 'WO']

Working on a subset:

In [45]:
# 1) get subset
identities_subset = identities["X*"]
identities_subset.names

['XEX',
 'XNATY',
 'XPOIL',
 'XPWMAB',
 'XPWMS',
 'XPWXAB',
 'XPWXS',
 'XQWXAB',
 'XQWXS',
 'XQWXSS',
 'XRLBER',
 'XTFP',
 'XW']

In [46]:
# 2) add an identity to the subset
identities_subset["XDPU"] = "grt DPU"
identities_subset["XDPU"]

Identity('grt DPU')

In [47]:
# --> new identity also appears in the global workspace
"XDPU" in identities, identities["XDPU"]

(True, Identity('grt DPU'))

In [48]:
# 3) update an identity in the subset
identities_subset["XDPU"] = "0"
identities_subset["XDPU"]

Identity('0')

In [49]:
# --> identity is also updated in the global workspace
identities["XDPU"]

Identity('0')

In [50]:
# delete one identity from a subset of the global database
identities_subset = identities["Y*"]
identities_subset.names

['Y', 'YSEFPR', 'YSFICR']

In [51]:
del identities_subset["Y"]
identities_subset.names

['YSEFPR', 'YSFICR']

In [52]:
# NOTE: the identity has also been deleted from the global database
"Y" in identities

False

In [53]:
identities.get_names("Y*")

['YSEFPR', 'YSFICR']

### Get - add - update - delete list(s)

Add one list:

In [54]:
# --- by passing a string
lists["A_VAR"] = "ACAF;ACAG;AOUC;AOUC_;AQC"
lists["A_VAR"]

['ACAF', 'ACAG', 'AOUC', 'AOUC_', 'AQC']

In [55]:
# --- by passing a Python list
b_vars = variables.get_names("B*")
b_vars

['BENEF', 'BQY', 'BRUGP', 'BVY']

In [56]:
lists["B_VAR"] = b_vars
lists["B_VAR"]

['BENEF', 'BQY', 'BRUGP', 'BVY']

Update a list:

In [57]:
# --- by passing a string
lists["A_VAR"] = "ACAF;ACAG;AOUC;AQC"
lists["A_VAR"]

['ACAF', 'ACAG', 'AOUC', 'AQC']

In [58]:
# --- by passing a Python list
b_y_vars = variables.get_names("B*Y")
b_y_vars

['BQY', 'BVY']

In [59]:
lists["B_VAR"] = b_y_vars
lists["B_VAR"]

['BQY', 'BVY']

Delete a list:

In [60]:
lists.get_names("C*")

['COPY', 'COPY0', 'COPY1']

In [61]:
del lists["COPY"]
lists.get_names("C*")

['COPY0', 'COPY1']

Working on a subset:

In [62]:
# 1) get subset
lists_subset = lists["E*"]
lists_subset.names

['ENDO', 'ENDO0', 'ENDO1', 'ENVI']

In [63]:
# 2) add a list to the subset
lists_subset["E_VAR"] = variables.get_names("E*")
lists_subset["E_VAR"]

['EFMY', 'EFXY', 'EX', 'EXC', 'EXCC', 'EXCCR']

In [64]:
# --> new list also appears in the global workspace
"E_VAR" in lists, lists["E_VAR"]

(True, ['EFMY', 'EFXY', 'EX', 'EXC', 'EXCC', 'EXCCR'])

In [65]:
# 3) update a list in the subset
lists_subset["E_VAR"] = "EX;EXC;EXCC;EXCCR"
lists_subset["E_VAR"]

['EX', 'EXC', 'EXCC', 'EXCCR']

In [66]:
# --> list is also updated in the global workspace
lists["E_VAR"]

['EX', 'EXC', 'EXCC', 'EXCCR']

In [67]:
# delete one IODE list from a subset of the global database
lists_subset = lists["E*"]
lists_subset.names

['ENDO', 'ENDO0', 'ENDO1', 'ENVI', 'E_VAR']

In [68]:
del lists_subset["ENVI"]
lists_subset.names

['ENDO', 'ENDO0', 'ENDO1', 'E_VAR']

In [69]:
# NOTE: the IODE list has also been deleted from the global database
"ENVI" in lists

False

In [70]:
lists.get_names("E*")

['ENDO', 'ENDO0', 'ENDO1', 'E_VAR']

### Get - add - update - delete scalar(s)

Add one scalar:

In [71]:
# 1. default relax to 1.0
scalars["a0"] = 0.1
scalars["a0"]

Scalar(0.1, 1, na)

In [72]:
# 2. value + relax
scalars["a1"] = 0.1, 0.9
scalars["a1"]

Scalar(0.1, 0.9, na)

Update a scalar:

In [73]:
scalars["acaf1"]

Scalar(0.0157684, 1, 0.00136871)

In [74]:
# only update the value
scalars["acaf1"] = 0.8
scalars["acaf1"]

Scalar(0.8, 1, 0.00136871)

In [75]:
# update value and relax (tuple)
scalars["acaf2"] = 0.8, 0.9
scalars["acaf2"]

Scalar(0.8, 0.9, na)

In [76]:
# update value and relax (list)
scalars["acaf2"] = (0.7, 0.8)
scalars["acaf2"]

Scalar(0.7, 0.8, na)

In [77]:
# update value and relax (dictionary)
scalars["acaf3"] = {"relax": 0.9, "value": 0.8}
scalars["acaf3"]

Scalar(0.8, 0.9, 0.87301)

In [78]:
# update value and/or relax (Scalar object)
# NOTE: the standard deviation (std) cannot be changed manually
scalars["acaf4"]

Scalar(-0.00850518, 1, 0.0020833)

In [79]:
scalars["acaf4"].value = 0.8
scalars["acaf4"].relax = 0.9
scalars["acaf4"]

Scalar(0.8, 0.9, 0.0020833)

Delete a scalar:

In [80]:
scalars.get_names("a*")

['a0', 'a1', 'acaf1', 'acaf2', 'acaf3', 'acaf4']

In [81]:
del scalars["acaf4"]
scalars.get_names("a*")

['a0', 'a1', 'acaf1', 'acaf2', 'acaf3']

Working on a subset:

In [82]:
# 1) get subset
scalars_subset = scalars["a*"]
scalars_subset.names

['a0', 'a1', 'acaf1', 'acaf2', 'acaf3']

In [83]:
# 2) add a scalar to the subset
scalars_subset["acaf0"] = 1.0, 1.0
scalars_subset["acaf0"]

Scalar(1, 1, na)

In [84]:
# --> new scalar also appears in the global workspace
"acaf0" in scalars, scalars["acaf0"]

(True, Scalar(1, 1, na))

In [85]:
# 3) update a scalar in the subset
scalars_subset["acaf0"] = 0.1
scalars_subset["acaf0"]

Scalar(0.1, 1, na)

In [86]:
# --> scalar is also updated in the global workspace
scalars["acaf0"]

Scalar(0.1, 1, na)

In [87]:
# delete one scalar from a subset of the global database
scalars_subset = scalars["z*"]
scalars_subset.names

['zkf1', 'zkf2', 'zkf3']

In [88]:
del scalars_subset["zkf2"]
scalars_subset.names

['zkf1', 'zkf3']

In [89]:
# NOTE: the scalar has also been deleted from the global database
"zkf2" in scalars

False

In [90]:
scalars.get_names("z*")

['zkf1', 'zkf3']

### Get - add - update - delete table(s)

Create an add a new table:

In [91]:
# 1. specify list of line titles and list of LEC expressions
lines_titles = ["GOSG:", "YDTG:", "DTH:", "DTF:", "IT:", "YSSG+COTRES:", "RIDG:", "OCUG:"]
lines_lecs = ["GOSG", "YDTG", "DTH", "DTF", "IT", "YSSG+COTRES", "RIDG", "OCUG"]
tables["TABLE_CELL_LECS"] = {"nb_columns": 2, "table_title": "New Table", "lecs_or_vars": lines_lecs,
                             "lines_titles": lines_titles, "mode": True, "files": True, "date": True}
tables["TABLE_CELL_LECS"]

DIVIS | 1              |            
TITLE |         "New Table"         
----- | ----------------------------
CELL  | ""             |     "#S"   
----- | ----------------------------
CELL  | "GOSG:"        |        GOSG
CELL  | "YDTG:"        |        YDTG
CELL  | "DTH:"         |         DTH
CELL  | "DTF:"         |         DTF
CELL  | "IT:"          |          IT
CELL  | "YSSG+COTRES:" | YSSG+COTRES
CELL  | "RIDG:"        |        RIDG
CELL  | "OCUG:"        |        OCUG
----- | ----------------------------
MODE  | 
FILES | 
DATE  | 

nb lines: 16
nb columns: 2
language: 'ENGLISH'
gridx: 'MAJOR'
gridy: 'MAJOR'
graph_axis: 'VALUES'
graph_alignment: 'LEFT'

In [92]:
# 2. specify list of variables
vars_list = ["GOSG", "YDTG", "DTH", "DTF", "IT", "YSSG", "COTRES", "RIDG", "OCUG", "$ENVI"]
tables["TABLE_VARS"] = {"nb_columns": 2, "table_title": "New Table", "lecs_or_vars": vars_list,
                        "mode": True, "files": True, "date": True}
tables["TABLE_VARS"]

DIVIS | 1  |     
TITLE | "New Table"
----- | -----------
CELL  | "" | "#S"
----- | -----------
----- | -----------
MODE  | 
FILES | 
DATE  | 

nb lines: 8
nb columns: 2
language: 'ENGLISH'
gridx: 'MAJOR'
gridy: 'MAJOR'
graph_axis: 'VALUES'
graph_alignment: 'LEFT'

Update a table:

In [93]:
tables["TABLE_CELL_LECS"]               

DIVIS | 1              |            
TITLE |         "New Table"         
----- | ----------------------------
CELL  | ""             |     "#S"   
----- | ----------------------------
CELL  | "GOSG:"        |        GOSG
CELL  | "YDTG:"        |        YDTG
CELL  | "DTH:"         |         DTH
CELL  | "DTF:"         |         DTF
CELL  | "IT:"          |          IT
CELL  | "YSSG+COTRES:" | YSSG+COTRES
CELL  | "RIDG:"        |        RIDG
CELL  | "OCUG:"        |        OCUG
----- | ----------------------------
MODE  | 
FILES | 
DATE  | 

nb lines: 16
nb columns: 2
language: 'ENGLISH'
gridx: 'MAJOR'
gridy: 'MAJOR'
graph_axis: 'VALUES'
graph_alignment: 'LEFT'

In [94]:
tables["TABLE_CELL_LECS"].graph_axis

'VALUES'

In [95]:
from iode import TableGraphAxis
# set graph axis type
tables["TABLE_CELL_LECS"].graph_axis = TableGraphAxis.SEMILOG
tables["TABLE_CELL_LECS"].graph_axis

'SEMILOG'

In [96]:
# get the first line
tables["TABLE_CELL_LECS"][0]

New Table

In [97]:
# get the last line
tables["TABLE_CELL_LECS"][-1]

<DATE>

In [98]:
# delete last line
del tables["TABLE_CELL_LECS"][-1]
tables["TABLE_CELL_LECS"]

DIVIS | 1              |            
TITLE |         "New Table"         
----- | ----------------------------
CELL  | ""             |     "#S"   
----- | ----------------------------
CELL  | "GOSG:"        |        GOSG
CELL  | "YDTG:"        |        YDTG
CELL  | "DTH:"         |         DTH
CELL  | "DTF:"         |         DTF
CELL  | "IT:"          |          IT
CELL  | "YSSG+COTRES:" | YSSG+COTRES
CELL  | "RIDG:"        |        RIDG
CELL  | "OCUG:"        |        OCUG
----- | ----------------------------
MODE  | 
FILES | 

nb lines: 15
nb columns: 2
language: 'ENGLISH'
gridx: 'MAJOR'
gridy: 'MAJOR'
graph_axis: 'SEMILOG'
graph_alignment: 'LEFT'

In [99]:
# get index of line containing YSSG+COTRES
index = tables["TABLE_CELL_LECS"].index("YSSG+COTRES")
index

9

In [100]:
tables["TABLE_CELL_LECS"][index]

('"YSSG+COTRES:"', 'YSSG+COTRES')

In [101]:
# get line type
tables["TABLE_CELL_LECS"][index].line_type

'CELL'

In [102]:
# get line graph type
tables["TABLE_CELL_LECS"][index].graph_type

'LINE'

In [103]:
# know if axis is left
tables["TABLE_CELL_LECS"][index].axis_left

True

In [104]:
# update cells
# double quotes "    -> STRING cell
# no double quotes   -> LEC cell
tables["TABLE_CELL_LECS"][index] = ('"YSSG:"', 'YSSG')
tables["TABLE_CELL_LECS"][index]

('"YSSG:"', 'YSSG')

In [105]:
# insert a new title line surrounded by two separator lines
tables["TABLE_CELL_LECS"].insert(index + 1, '-')
tables["TABLE_CELL_LECS"].insert(index + 2, "New Title")
tables["TABLE_CELL_LECS"].insert(index + 3, '-')
tables["TABLE_CELL_LECS"]

DIVIS | 1       |     
TITLE |  "New Table"  
----- | --------------
CELL  | ""      | "#S"
----- | --------------
CELL  | "GOSG:" | GOSG
CELL  | "YDTG:" | YDTG
CELL  | "DTH:"  |  DTH
CELL  | "DTF:"  |  DTF
CELL  | "IT:"   |   IT
CELL  | "YSSG:" | YSSG
----- | --------------
TITLE |  "New Title"  
----- | --------------
CELL  | "RIDG:" | RIDG
CELL  | "OCUG:" | OCUG
----- | --------------
MODE  | 
FILES | 

nb lines: 18
nb columns: 2
language: 'ENGLISH'
gridx: 'MAJOR'
gridy: 'MAJOR'
graph_axis: 'SEMILOG'
graph_alignment: 'LEFT'

In [106]:
# append a new sepatator line
tables["TABLE_CELL_LECS"] += '-'
tables["TABLE_CELL_LECS"]

DIVIS | 1       |     
TITLE |  "New Table"  
----- | --------------
CELL  | ""      | "#S"
----- | --------------
CELL  | "GOSG:" | GOSG
CELL  | "YDTG:" | YDTG
CELL  | "DTH:"  |  DTH
CELL  | "DTF:"  |  DTF
CELL  | "IT:"   |   IT
CELL  | "YSSG:" | YSSG
----- | --------------
TITLE |  "New Title"  
----- | --------------
CELL  | "RIDG:" | RIDG
CELL  | "OCUG:" | OCUG
----- | --------------
MODE  | 
FILES | 
----- | --------------

nb lines: 19
nb columns: 2
language: 'ENGLISH'
gridx: 'MAJOR'
gridy: 'MAJOR'
graph_axis: 'SEMILOG'
graph_alignment: 'LEFT'

Delete a table:

In [107]:
tables.get_names("G*")

['GAP', 'GDP', 'GFR', 'GFRLEVEL', 'GFRPC', 'GROWTH']

In [108]:
del tables["GFRLEVEL"]
tables.get_names("G*")

['GAP', 'GDP', 'GFR', 'GFRPC', 'GROWTH']

Working on a subset:

In [109]:
# 1) get subset
tables_subset = tables["C8_?"]
tables_subset.names

['C8_1', 'C8_2', 'C8_3', 'C8_4', 'C8_5', 'C8_6', 'C8_7', 'C8_8', 'C8_9']

In [110]:
# 2) add a table to the subset
vars_list = ["XNATY", "XPOIL", "XPWMAB", "XPWXAB"]
tables_subset["X_GRT"] = {"nb_columns": 2, "table_title": "Croissance", "lecs_or_vars": vars_list,
                          "mode": True, "files": True, "date": True}
tables_subset["X_GRT"]

DIVIS | 1                                                       |       
TITLE |                           "Croissance"                          
----- | ----------------------------------------------------------------
CELL  | ""                                                      |  "#S" 
----- | ----------------------------------------------------------------
CELL  | "Croissance de la population active"                    |  XNATY
CELL  | "Croissance du prix du pétrole"                         |  XPOIL
CELL  | "Croissance des prix des biens importés"                | XPWMAB
CELL  | "Croissance des prix des marchés pertinents à l'export" | XPWXAB
----- | ----------------------------------------------------------------
MODE  | 
FILES | 
DATE  | 

nb lines: 12
nb columns: 2
language: 'ENGLISH'
gridx: 'MAJOR'
gridy: 'MAJOR'
graph_axis: 'VALUES'
graph_alignment: 'LEFT'

In [111]:
# --> new table also appears in the global workspace
"X_GRT" in tables, tables["X_GRT"]

(True,
 DIVIS | 1                                                       |       
 TITLE |                           "Croissance"                          
 ----- | ----------------------------------------------------------------
 CELL  | ""                                                      |  "#S" 
 ----- | ----------------------------------------------------------------
 CELL  | "Croissance de la population active"                    |  XNATY
 CELL  | "Croissance du prix du pétrole"                         |  XPOIL
 CELL  | "Croissance des prix des biens importés"                | XPWMAB
 CELL  | "Croissance des prix des marchés pertinents à l'export" | XPWXAB
 ----- | ----------------------------------------------------------------
 MODE  | 
 FILES | 
 DATE  | 
 
 nb lines: 12
 nb columns: 2
 language: 'ENGLISH'
 gridx: 'MAJOR'
 gridy: 'MAJOR'
 graph_axis: 'VALUES'
 graph_alignment: 'LEFT')

In [112]:
# 3) update a table in the subset
table_x_grt = tables_subset["X_GRT"]
table_x_grt

DIVIS | 1                                                       |       
TITLE |                           "Croissance"                          
----- | ----------------------------------------------------------------
CELL  | ""                                                      |  "#S" 
----- | ----------------------------------------------------------------
CELL  | "Croissance de la population active"                    |  XNATY
CELL  | "Croissance du prix du pétrole"                         |  XPOIL
CELL  | "Croissance des prix des biens importés"                | XPWMAB
CELL  | "Croissance des prix des marchés pertinents à l'export" | XPWXAB
----- | ----------------------------------------------------------------
MODE  | 
FILES | 
DATE  | 

nb lines: 12
nb columns: 2
language: 'ENGLISH'
gridx: 'MAJOR'
gridy: 'MAJOR'
graph_axis: 'VALUES'
graph_alignment: 'LEFT'

In [113]:
index = table_x_grt.index("XPWXAB")
table_x_grt.insert(index + 1, (f'"{comments["XQWXSS"]}"', "XQWXSS"))
tables_subset["X_GRT"]                      

DIVIS | 1                                                       |       
TITLE |                           "Croissance"                          
----- | ----------------------------------------------------------------
CELL  | ""                                                      |  "#S" 
----- | ----------------------------------------------------------------
CELL  | "Croissance de la population active"                    |  XNATY
CELL  | "Croissance du prix du pétrole"                         |  XPOIL
CELL  | "Croissance des prix des biens importés"                | XPWMAB
CELL  | "Croissance des prix des marchés pertinents à l'export" | XPWXAB
CELL  | "Croissance des marchés pertinents"                     | XQWXSS
----- | ----------------------------------------------------------------
MODE  | 
FILES | 
DATE  | 

nb lines: 13
nb columns: 2
language: 'ENGLISH'
gridx: 'MAJOR'
gridy: 'MAJOR'
graph_axis: 'VALUES'
graph_alignment: 'LEFT'

In [114]:
# --> table is also updated in the global workspace
tables["X_GRT"]                       

DIVIS | 1                                                       |       
TITLE |                           "Croissance"                          
----- | ----------------------------------------------------------------
CELL  | ""                                                      |  "#S" 
----- | ----------------------------------------------------------------
CELL  | "Croissance de la population active"                    |  XNATY
CELL  | "Croissance du prix du pétrole"                         |  XPOIL
CELL  | "Croissance des prix des biens importés"                | XPWMAB
CELL  | "Croissance des prix des marchés pertinents à l'export" | XPWXAB
CELL  | "Croissance des marchés pertinents"                     | XQWXSS
----- | ----------------------------------------------------------------
MODE  | 
FILES | 
DATE  | 

nb lines: 13
nb columns: 2
language: 'ENGLISH'
gridx: 'MAJOR'
gridy: 'MAJOR'
graph_axis: 'VALUES'
graph_alignment: 'LEFT'

In [115]:
# delete one table from a subset of the global database
tables_subset = tables["M*"]
tables_subset.names

['MULT1FR', 'MULT1RESU', 'MULT2FR', 'MULT2RESU']

In [116]:
del tables_subset["MULT2RESU"]
tables_subset.names

['MULT1FR', 'MULT1RESU', 'MULT2FR']

In [117]:
# NOTE: the table has also been deleted from the global database
"MULT2RESU" in tables

False

In [118]:
tables.get_names("M*")

['MULT1FR', 'MULT1RESU', 'MULT2FR']

### Get - add - update - delete variable(s)

Add one variable:

In [119]:
# 1) same value for all periods
variables["A0"] = np.nan
variables["A0"]

[nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan]

In [120]:
# 2) vector (list) containing a specific value for each period
variables["A1"] = list(range(variables.nb_periods))
variables["A1"]                     

[0.0,
 1.0,
 2.0,
 3.0,
 4.0,
 5.0,
 6.0,
 7.0,
 8.0,
 9.0,
 10.0,
 11.0,
 12.0,
 13.0,
 14.0,
 15.0,
 16.0,
 17.0,
 18.0,
 19.0,
 20.0,
 21.0,
 22.0,
 23.0,
 24.0,
 25.0,
 26.0,
 27.0,
 28.0,
 29.0,
 30.0,
 31.0,
 32.0,
 33.0,
 34.0,
 35.0,
 36.0,
 37.0,
 38.0,
 39.0,
 40.0,
 41.0,
 42.0,
 43.0,
 44.0,
 45.0,
 46.0,
 47.0,
 48.0,
 49.0,
 50.0,
 51.0,
 52.0,
 53.0,
 54.0,
 55.0]

In [121]:
# 3) LEC expression
variables["A2"] = "t + 10"
variables["A2"]                     

[10.0,
 11.0,
 12.0,
 13.0,
 14.0,
 15.0,
 16.0,
 17.0,
 18.0,
 19.0,
 20.0,
 21.0,
 22.0,
 23.0,
 24.0,
 25.0,
 26.0,
 27.0,
 28.0,
 29.0,
 30.0,
 31.0,
 32.0,
 33.0,
 34.0,
 35.0,
 36.0,
 37.0,
 38.0,
 39.0,
 40.0,
 41.0,
 42.0,
 43.0,
 44.0,
 45.0,
 46.0,
 47.0,
 48.0,
 49.0,
 50.0,
 51.0,
 52.0,
 53.0,
 54.0,
 55.0,
 56.0,
 57.0,
 58.0,
 59.0,
 60.0,
 61.0,
 62.0,
 63.0,
 64.0,
 65.0]

Update a variable:

In [122]:
# 1) update all values of a Variable
variables["ACAF"]                   

[nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 1.2130001,
 5.2020001,
 9.184,
 8.0790005,
 11.332,
 13.518001,
 15.784,
 16.544001,
 21.489,
 20.281,
 21.277,
 32.417999,
 24.446999,
 27.025002,
 24.504,
 27.560999,
 25.542,
 27.499001,
 25.353001,
 17.165001,
 23.771,
 26.240999,
 30.159,
 34.661999,
 8.1610022,
 -13.130997,
 32.171001,
 39.935799,
 29.645657,
 13.530404919696034,
 10.046610792200543,
 2.8679227364554634,
 -0.9292125090516451,
 -6.091564987568884,
 -14.582094462898064,
 -26.538789569788573,
 -28.987287982597536,
 -33.378425784295416,
 -38.40951778239742,
 -37.46350964127375,
 -37.82742883229439,
 -44.544792633543224,
 -55.55928981721873,
 -68.89465432262006,
 -83.34062511080091,
 -96.41041982848331]

In [123]:
# 1.I) same value for all periods
variables["ACAF"] = np.nan
variables["ACAF"]                   

[nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan]

In [124]:
# 1.II) vector (list) containing a specific value for each period
variables["ACAF"] = list(range(variables.nb_periods))
variables["ACAF"]                   

[0.0,
 1.0,
 2.0,
 3.0,
 4.0,
 5.0,
 6.0,
 7.0,
 8.0,
 9.0,
 10.0,
 11.0,
 12.0,
 13.0,
 14.0,
 15.0,
 16.0,
 17.0,
 18.0,
 19.0,
 20.0,
 21.0,
 22.0,
 23.0,
 24.0,
 25.0,
 26.0,
 27.0,
 28.0,
 29.0,
 30.0,
 31.0,
 32.0,
 33.0,
 34.0,
 35.0,
 36.0,
 37.0,
 38.0,
 39.0,
 40.0,
 41.0,
 42.0,
 43.0,
 44.0,
 45.0,
 46.0,
 47.0,
 48.0,
 49.0,
 50.0,
 51.0,
 52.0,
 53.0,
 54.0,
 55.0]

In [125]:
# 1.III) LEC expression
variables["ACAF"] = "t + 10"
variables["ACAF"]                   

[10.0,
 11.0,
 12.0,
 13.0,
 14.0,
 15.0,
 16.0,
 17.0,
 18.0,
 19.0,
 20.0,
 21.0,
 22.0,
 23.0,
 24.0,
 25.0,
 26.0,
 27.0,
 28.0,
 29.0,
 30.0,
 31.0,
 32.0,
 33.0,
 34.0,
 35.0,
 36.0,
 37.0,
 38.0,
 39.0,
 40.0,
 41.0,
 42.0,
 43.0,
 44.0,
 45.0,
 46.0,
 47.0,
 48.0,
 49.0,
 50.0,
 51.0,
 52.0,
 53.0,
 54.0,
 55.0,
 56.0,
 57.0,
 58.0,
 59.0,
 60.0,
 61.0,
 62.0,
 63.0,
 64.0,
 65.0]

In [126]:
# 2) set one value of a Variable for a specific period
variables["ACAG", "1990Y1"]

-28.1721855713507

In [127]:
variables["ACAG", "1990Y1"] = -28.2
variables["ACAG", "1990Y1"]

-28.2

In [128]:
# 3) set the variable values for range of periods
# 3.I) using a Python slice
# 3.I.a) variable(periods) = same value for all periods
variables["ACAF", "1991Y1":"1995Y1"] = 0.0
variables["ACAF", "1991Y1":"1995Y1"]

[0.0, 0.0, 0.0, 0.0, 0.0]

In [129]:
# 3.I.b) variable(periods) = vector (list) containing a specific value for each period
variables["ACAF", "1991Y1":"1995Y1"] = [0., 1., 2., 3., 4.]
variables["ACAF", "1991Y1":"1995Y1"]

[0.0, 1.0, 2.0, 3.0, 4.0]

In [130]:
# 3.I.c) variable(periods) = LEC expression
variables["ACAF", "1991Y1":"1995Y1"] = "t + 10"
variables["ACAF", "1991Y1":"1995Y1"]

[41.0, 42.0, 43.0, 44.0, 45.0]

In [131]:
# 3.II) same as above but with the colon ':' inside the periods range string
# 3.II.a) variable(periods) = same value for all periods
variables["ACAF", "1991Y1:1995Y1"] = 0.0
variables["ACAF", "1991Y1:1995Y1"]

[0.0, 0.0, 0.0, 0.0, 0.0]

In [132]:
# 3.II.b) variable(periods) = vector (list) containing a specific value for each period
variables["ACAF", "1991Y1:1995Y1"] = [0., -1., -2., -3., -4.]
variables["ACAF", "1991Y1":"1995Y1"]

[0.0, -1.0, -2.0, -3.0, -4.0]

In [133]:
# 3.II.c) variable(periods) = LEC expression
variables["ACAF", "1991Y1:1995Y1"] = "t - 10"
variables["ACAF", "1991Y1:1995Y1"]

[21.0, 22.0, 23.0, 24.0, 25.0]

Delete a variable:

In [134]:
variables.get_names("A*")

['A0', 'A1', 'A2', 'ACAF', 'ACAG', 'AOUC', 'AOUC_', 'AQC']

In [135]:
del variables["ACAF"]
variables.get_names("A*")

['A0', 'A1', 'A2', 'ACAG', 'AOUC', 'AOUC_', 'AQC']

Working on a subset:

In [136]:
# 1) get subset
variables_subset = variables["A*"]
variables_subset.names

['A0', 'A1', 'A2', 'ACAG', 'AOUC', 'AOUC_', 'AQC']

In [137]:
# 2) add a variable to the subset
variables_subset["A3"] = np.nan
variables_subset["A3"]              

[nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan]

In [138]:
# --> new variable also appears in the global workspace
"A3" in variables, variables["A3"]            

(True,
 [nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan])

In [139]:
# 3) update a variable in the subset
variables_subset["A3"] = 0.0
variables_subset["A3"]              

[0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0]

In [140]:
# --> variable is also updated in the global workspace
variables["A3"]                     

[0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0]

In [141]:
# delete one variable from a subset of the global database
variables_subset = variables["D*"]
variables_subset.names

['DEBT',
 'DPU',
 'DPUF',
 'DPUG',
 'DPUGO',
 'DPUH',
 'DPUHO',
 'DPUU',
 'DTF',
 'DTFX',
 'DTH',
 'DTH1',
 'DTH1C',
 'DTHX']

In [142]:
del variables_subset["DPUGO"]
variables_subset.names

['DEBT',
 'DPU',
 'DPUF',
 'DPUG',
 'DPUH',
 'DPUHO',
 'DPUU',
 'DTF',
 'DTFX',
 'DTH',
 'DTH1',
 'DTH1C',
 'DTHX']

In [143]:
# NOTE: the variable has also been deleted from the global database
"DPUGO" in variables

False

In [144]:
variables.get_names("D*")

['DEBT',
 'DPU',
 'DPUF',
 'DPUG',
 'DPUH',
 'DPUHO',
 'DPUU',
 'DTF',
 'DTFX',
 'DTH',
 'DTH1',
 'DTH1C',
 'DTHX']

## Estimation

To estimate either one equation or a block of equations, use the [Equation.estimate](../_generated/iode.Equation.estimate.rst#iode.Equation.estimate) method of the equation object.

In [145]:
help(equations.estimate)

Help on method estimate in module iode.iode_cython:

estimate(from_period: 'Union[str, Period]' = None, to_period: 'Union[str, Period]' = None, list_eqs: 'Union[str, List[str]]' = None) method of iode.iode_cython.Equations instance
    estimate(self, from_period: Union[str, Period] = None, to_period: Union[str, Period] = None, list_eqs: Union[str, List[str]] = None)
    
    Estimate an equation or a block of equations.
    
    At the end of the estimation process, certain variables and scalars are automatically created 
    if the process has converged. These variables and scalars can be used for computational purposes and, 
    as they are part of the global workspace, can be saved for future use.
    
    The tests resulting from the last estimation are saved as scalars. The same applies to residuals, 
    left-hand and right-hand members of equations.
    
    Saved tests (as scalars) have the following names (`e<ith>_*` for the <ith> equation of the block):
    
        - `e<ith>

Let's start by reloading all workspaces:

In [146]:
comments.load(f"{SAMPLE_DATA_DIR}/fun.cmt")
equations.load(f"{SAMPLE_DATA_DIR}/fun.eqs")
identities.load(f"{SAMPLE_DATA_DIR}/fun.idt")
lists.load(f"{SAMPLE_DATA_DIR}/fun.lst")
scalars.load(f"{SAMPLE_DATA_DIR}/fun.scl")
tables.load(f"{SAMPLE_DATA_DIR}/fun.tbl")
variables.load(f"{SAMPLE_DATA_DIR}/fun.var")

Loading C:\soft\Miniconda3\Lib\site-packages\iode\tests\data/fun.cmt
317 objects loaded
Loading C:\soft\Miniconda3\Lib\site-packages\iode\tests\data/fun.eqs
274 objects loaded
Loading C:\soft\Miniconda3\Lib\site-packages\iode\tests\data/fun.idt
48 objects loaded
Loading C:\soft\Miniconda3\Lib\site-packages\iode\tests\data/fun.lst
17 objects loaded
Loading C:\soft\Miniconda3\Lib\site-packages\iode\tests\data/fun.scl
161 objects loaded
Loading C:\soft\Miniconda3\Lib\site-packages\iode\tests\data/fun.tbl
46 objects loaded
Loading C:\soft\Miniconda3\Lib\site-packages\iode\tests\data/fun.var
394 objects loaded


Example for one equation:

In [147]:
# ---- estimate coefficients of one equation ----
print(f"ACAF equation LEC: {equations['ACAF'].lec}")
print(f"ACAF equations coefficients (= scalars): {equations['ACAF'].coefficients}")
print(f"ACAF equations variables: {equations['ACAF'].variables}")

# copy the original values of the coefficients into an isolated scalars workspace
# Useful to restore the original values if the estimation process didn't go well
scalars_copy = scalars[equations['ACAF'].coefficients].copy()
print(f"scalars names in scalars_copy: {scalars_copy.names}")

# reset scalars
for name in equations['ACAF'].coefficients:
    scalars[name] = 0., 1.

# estimate the 'ACAF' equation for the periods ranging from '1980Y1' to '1996Y1'
equations.estimate("1980Y1", "1996Y1", "ACAF")
# or equivalently
equations["ACAF"].estimate("1980Y1", "1996Y1")

print(f"Resulting values for the coefficient 'acaf1': {scalars['acaf1']}")
print(f"Resulting values for the coefficient 'acaf2': {scalars['acaf2']}")
print(f"Resulting values for the coefficient 'acaf4': {scalars['acaf4']}")

ACAF equation LEC: (ACAF/VAF[-1]) :=acaf1+acaf2*GOSF[-1]+
acaf4*(TIME=1995)
ACAF equations coefficients (= scalars): ['acaf1', 'acaf2', 'acaf4']
ACAF equations variables: ['ACAF', 'VAF', 'GOSF', 'TIME']
scalars names in scalars_copy: ['acaf1', 'acaf2', 'acaf4']
Estimating : iteration 1 (||eps|| = 0.173205)

Estimating : iteration 2 (||eps|| = 5.16075e-09)

Solution reached after 2 iteration(s). Creating results file ...

Estimating : iteration 1 (||eps|| = 7.05003e-13)

Solution reached after 1 iteration(s). Creating results file ...

Resulting values for the coefficient 'acaf1': Scalar(0.0157705, 1, 0.00136949)
Resulting values for the coefficient 'acaf2': Scalar(-7.96505e-06, 1, 1.48247e-06)
Resulting values for the coefficient 'acaf4': Scalar(-0.0085027, 1, 0.00208257)


Example for a block of equations:

In [148]:
# ---- estimate a block of equations ----
print(f"ACAF equation LEC: {equations['ACAF'].lec}")
print(f"ACAF equations coefficients (= scalars): {equations['ACAF'].coefficients}")
print(f"ACAF equations variables: {equations['ACAF'].variables}")
print(f"DPUH equation LEC: {equations['DPUH'].lec}")
print(f"DPUH equations coefficients (= scalars): {equations['DPUH'].coefficients}")
print(f"DPUH equations variables: {equations['DPUH'].variables}")

# copy the original values of the coefficients into an isolated scalars workspace
# Useful to restore the original values if the estimation process didn't go well
scalars_names = equations['ACAF'].coefficients + equations['DPUH'].coefficients
scalars_names = list(set(scalars_names))   # removed duplicates
scalars_copy = scalars[scalars_names].copy()
print(f"scalars names in scalars_copy: {scalars_copy.names}")

# reset scalars
for name in equations['ACAF'].coefficients:
    scalars[name] = 0., 1.
for name in equations['DPUH'].coefficients:
    scalars[name] = 0., 1.

# prepare equations (same block and method)
block = "ACAF;DPUH"
for name in block.split(";"):
    equations[name] = {"block": block, "method": "LSQ"}

# estimation the block 'ACAF;DPUH' for the periods ranging from '1980Y1' to '1996Y1'
equations.estimate("1980Y1", "1996Y1", block)

print(f"Resulting values for the coefficient 'acaf1': {scalars['acaf1']}")
print(f"Resulting values for the coefficient 'acaf2': {scalars['acaf2']}")
print(f"Resulting values for the coefficient 'acaf4': {scalars['acaf4']}")
print(f"Resulting values for the coefficient 'dpuh_1': {scalars['dpuh_1']}")
print(f"Resulting values for the coefficient 'dpuh_2': {scalars['dpuh_2']}")

ACAF equation LEC: (ACAF/VAF[-1]) :=acaf1+acaf2*GOSF[-1]+
acaf4*(TIME=1995)
ACAF equations coefficients (= scalars): ['acaf1', 'acaf2', 'acaf4']
ACAF equations variables: ['ACAF', 'VAF', 'GOSF', 'TIME']
DPUH equation LEC: dln (DPUH/DPUHO):=dpuh_1+dpuh_2*dln(IHU/PI5)+dln PC
DPUH equations coefficients (= scalars): ['dpuh_1', 'dpuh_2']
DPUH equations variables: ['DPUH', 'DPUHO', 'IHU', 'PI5', 'PC']
scalars names in scalars_copy: ['acaf1', 'acaf2', 'acaf4', 'dpuh_1', 'dpuh_2']
Estimating : iteration 1 (||eps|| = 0.223607)

Estimating : iteration 2 (||eps|| = 5.16075e-09)

Solution reached after 2 iteration(s). Creating results file ...

Estimating : iteration 1 (||eps|| = 4.38565e-12)

Solution reached after 1 iteration(s). Creating results file ...

Resulting values for the coefficient 'acaf1': Scalar(0.0157705, 1, 0.00136079)
Resulting values for the coefficient 'acaf2': Scalar(-7.96505e-06, 1, 1.47188e-06)
Resulting values for the coefficient 'acaf4': Scalar(-0.0085027, 1, 0.00206603)


## Simulation

To simulate a model, you must create and initialize an instance of the [Simulation](../_generated/iode.Simulation.rst#iode.Simulation) class:

In [149]:
from iode import Simulation

help(Simulation)

Help on class Simulation in module iode.iode_cython:

class Simulation(builtins.object)
 |  Class for simulate models.
 |  
 |  Models
 |  ~~~~~~
 |  
 |  IN IODE, a model is simply a list of equations. No other construction is necessary: in this way, 
 |  to modify a model, the only thing to do is to modify the list that defines it. 
 |  For example, if a model is logically divided into 5 blocks, 5 lists of equations will be defined:: 
 |  
 |      BLOC1 : A, B, C, D
 |      BLOC2 : X, Y
 |      BLOC3 : C1, C2, C3, C4
 |      BLOC4 : X_A1, X_A2, X_A3
 |      BLOC5 : R1, R2, R3, S_12
 |      MODSIM : $BLOC1, $BLOC2, $BLOC3, $BLOC4, $BLOC5
 |  
 |  To simulate a model, all equations of the model must have been loaded or created. In addition, 
 |  all the variables and scalars used in the model equations must have been loaded or created. 
 |  Values of exogenous variables and of scalar cannot be :math:`NA` (*Not Available*) over the 
 |  simulation periods.
 |  
 |  Exchange Endo-Exo
 | 

In [150]:
from iode import SimulationSort

print(f"possible sort algorithms: {[member.name for member in SimulationSort]}")

possible sort algorithms: ['CONNEX', 'BOTH', 'NONE']


In [151]:
from iode import SimulationInitialization

print(f"possible initialization methods: {[member.name for member in SimulationInitialization]}")

possible initialization methods: ['TM1', 'TM1_A', 'EXTRA', 'EXTRA_A', 'ASIS', 'TM1_NA', 'EXTRA_NA']


In [152]:
# ---- simulation ----
print("Simulation() methods and properties:")
for name in dir(Simulation):
    if not name.startswith('_'):
        print(name)

Simulation() methods and properties:
convergence_threshold
debug
debug_newton
initialization_method
max_nb_iterations
max_nb_iterations_newton
model_calculate_SCC
model_compile
model_exchange
model_simulate
model_simulate_SCC
nb_passes
relax
sort_algorithm


In [153]:
# create and initialize a Simulation instance
simu = Simulation(sort_algorithm=SimulationSort.BOTH, initialization_method=SimulationInitialization.TM1)
print(f"Simulation convergence_threshold: {simu.convergence_threshold}")
print(f"Simulation relax: {simu.relax}")
print(f"Simulation max_nb_iterations: {simu.max_nb_iterations}")
print(f"Simulation sort_algorithm: {simu.sort_algorithm}")
print(f"Simulation initialization_method: {simu.initialization_method}")
print(f"Simulation debug: {simu.debug}")
print(f"Simulation nb_passes: {simu.nb_passes}")
print(f"Simulation debug_newton: {simu.debug_newton}")

Simulation convergence_threshold: 0.001
Simulation relax: 1.0
Simulation max_nb_iterations: 100
Simulation sort_algorithm: BOTH (Connex compon. + Triangulation)
Simulation initialization_method: TM1 (Y := Y[-1], if Y null or NA)
Simulation debug: False
Simulation nb_passes: 5
Simulation debug_newton: False


To run the simulation, call the [Simulation.model_simulate](../_generated/iode.Simulation.model_simulate.rst#iode.Simulation.model_simulate) method:

In [154]:
help(Simulation.model_simulate)

Help on cython_function_or_method in module iode.iode_cython:

model_simulate(self, from_period: 'Union[str, Period]', to_period: 'Union[str, Period]', list_eqs: 'Union[str, List[str]]' = None)
    model_simulate(self, from_period: Union[str, Period], to_period: Union[str, Period], list_eqs: Union[str, List[str]] = None)
    
    Run the simulation of a model for a given sample.
    
    To simulate a model, all equations of the model must have been loaded or created. In addition, 
    all the variables and scalars used in the model equations must have been loaded or created. 
    Values of exogenous variables and of scalar cannot be :math:`NA` (*Not Available*) over the 
    simulation sample.
    
    Parameters
    ----------
    from_period : str or Period
        The starting period for the simulation.
    to_period : str or Period
        The ending period for the simulation.
    list_eqs : str or list(str), optional
        List of equations representing the model.
        Defau

In [155]:
print(f"exogenous variable 'UY': {equations['UY'].lec}")
print(f"endogenous variable 'XNATY': {identities['XNATY']}")
# reset values of exogenous variable
variables["UY", "2000Y1:2015Y1"] = 0.0

print(f"exogenous variable 'UY' before simulation:\n{variables['UY', '1998Y1:2005Y1']}")

# run the simulation for the periods range '2000Y1' to '2015Y1'
simu.model_simulate("2000Y1", "2015Y1")

print(f"exogenous variable 'UY' after simulation:\n{variables['UY', '1998Y1:2005Y1']}")

exogenous variable 'UY': UY := NATY-NDOMY-NIY-NGY-(EFXY-EFMY)-NFY
endogenous variable 'XNATY': grt NATY
exogenous variable 'UY' before simulation:
[671.46, 647.3906678388862, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
Linking equations ....
Calculating SCC...
Calculating SCC... 0 ms -> #PRE 31 - #INTER 204 - #POST 39
Reordering interdependent block...
Reordering interdependent block... 0 ms
2000Y1: 1 iters - error =    1.342 - cpu=0ms
2000Y1: 2 iters - error =   0.4115 - cpu=0ms
2000Y1: 3 iters - error =   0.5272 - cpu=0ms
2000Y1: 4 iters - error =   0.0215 - cpu=0ms
2000Y1: 5 iters - error =  0.03879 - cpu=0ms
2000Y1: 6 iters - error =  0.02778 - cpu=0ms
2000Y1: 7 iters - error =   0.1455 - cpu=0ms
2000Y1: 8 iters - error =  0.02579 - cpu=0ms
2000Y1: 9 iters - error =   0.1056 - cpu=0ms
2000Y1: 10 iters - error =  0.08991 - cpu=0ms
2000Y1: 11 iters - error =  0.02125 - cpu=0ms
2000Y1: 12 iters - error =  0.03417 - cpu=0ms
2000Y1: 13 iters - error =  0.03339 - cpu=0ms
2000Y1: 14 iters - error =  0.

## Import/Export IODE workspaces to pandas Series and DataFrame

To import / export the content of the `comments`, `identities` and `lists` workspaces to a pandas Series object, use the [from_series()](../_generated/iode.Comments.from_series.rst#iode.Comments.from_series) and [to_series()](../_generated/iode.Comments.to_series.rst#iode.Comments.to_series) methods. 

Alternatively, you can use the [series](../_generated/iode.Comments.series.rst#iode.Comments.series) property to export the content of the `comments`, `identities` and `lists` workspaces to a pandas Series object.

In [156]:
import pandas as pd

# ---- to pandas Series ----
# See Comments/Identities/Lists.to_series and 
#     Comments/Identities/Lists.series

series_cmt = comments.to_series()
print(f"Comments as pandas Series:\n{series_cmt.info}")
print()

series_idt = identities.to_series()
print(f"Identities as pandas Series:\n{series_idt.info}")
print()

series_lst = lists.to_series()
print(f"Lists as pandas Series:\n{series_lst.info}")

# Alternatively

series_cmt = comments.series
series_idt = identities.series
series_lst = lists.series

# ---- from pandas Series ----
# See Comments/Identities/Lists.from_series

comments.from_series(series_cmt)
identities.from_series(series_idt)
lists.from_series(series_lst)

Comments as pandas Series:
<bound method Series.info of ACAF        Ondernemingen: ontvangen kapitaaloverdrachten.
ACAG     Totale overheid: netto ontvangen kapitaaloverd...
AOUC                             Kost per eenheid produkt.
AQC          Kost per eenheid produkt: kapitaal en arbeid.
BENEF             Ondernemingen: niet-uitgekeerde winsten.
                               ...                        
ZF       Indexeringscoëfficiënt voor de lonen in de pri...
ZJ       Indexeringscoëfficiënt voor de sociale uitkeri...
ZKF            Bezettingsgraad van de produktiecapaciteit.
ZX                          Saut d'index (correction en %)
ZZ_      Marktsector (ondernemingen en zelfstandigen): ...
Name: Comments, Length: 317, dtype: object>

Identities as pandas Series:
<bound method Series.info of AOUC      ((WCRH/QL)/(WCRH/QL)[1990Y1])*(VAFF/(VM+VAFF))...
AOUC_     exp(ln(((WCF/NFYH)/QL)+PKF/(QAFF/KNFFY))*(QAFF...
FLGR                                               FLG/VBBP
GAP2        

To import / export the content of the `equations`, `scalars` and `variables` workspaces to a pandas DataFrame object, use the [from_frame()](../_generated/iode.Variables.from_frame.rst#iode.Variables.from_frame) and [to_frame()](../_generated/iode.Variables.to_frame.rst#iode.Variables.to_frame) methods. 

Alternatively, you can use the [df](../_generated/iode.Variables.df.rst#iode.Variables.df) property to export the content of the `equations`, `scalars` and `variables` workspaces to a pandas DataFrame object.

In [157]:
# ---- to pandas DataFrame ----
# See Equations/Scalars/Variables.to_frame and
#     Equations/Scalars/Variables.df

df_eqs = equations.to_frame()
print(f"Equations as pandas DataFrame:\n{df_eqs.info}")
print()

df_scl = scalars.to_frame()
print(f"Scalars as pandas DataFrame:\n{df_scl.info}")
print()

df_vars = variables.to_frame()
print(f"Variables as pandas DataFrame:\n{df_vars.info}")

# Alternatively

df_eqs = equations.df
df_scl = scalars.df
df_vars = variables.df

# ---- from pandas DataFrame ----
# See Equations/Scalars/Variables.from_frame

equations.from_frame(df_eqs)
scalars.from_frame(df_scl)
variables.from_frame(df_vars)

Equations as pandas DataFrame:
<bound method DataFrame.info of                                                      lec method  \
ACAF   (ACAF/VAF[-1]) :=acaf1+acaf2*GOSF[-1]+\nacaf4*...    LSQ   
ACAG   ACAG := ACAG[-1]+r VBBP[-1]+(0.006*VBBP[-1]*(T...    LSQ   
AOUC   AOUC:=((WCRH/QL)/(WCRH/QL)[1990Y1])*(VAFF/(VM+...    LSQ   
BENEF  d BENEF :=d(VBNP-(IT+ITCEE)+SUB-DPUU-(WCF+SSFF...    LSQ   
BQY                                    BQY:=(YK+YN)/PBBP    LSQ   
...                                                  ...    ...   
YSSF                                  dln YSSF:=dln WBF_    LSQ   
YSSG                       YSSG := SSF+SSH-(YSSF+COTRES)    LSQ   
ZF                       grt ZF :=grt PC+ZX-0.05*grt PME    LSQ   
ZJ                      grt ZJ :=grt PC +ZX-0.05*grt PME    LSQ   
ZZF_                                    ZZF_ := ZZF_[-1]    LSQ   

              sample comment instruments      block  corr    stdev     meany  \
ACAF   1980Y1:1996Y1                      ACAF;DPUH 

C:\Users\ALD\AppData\Local\Temp\ipykernel_11644\1615955290.py:5: UserWarning: 'sample' is not defined
  df_eqs = equations.to_frame()
C:\Users\ALD\AppData\Local\Temp\ipykernel_11644\1615955290.py:18: UserWarning: 'sample' is not defined
  df_eqs = equations.df


## Import/Export the Variables workspace to LArray

To import / export the content of the `variables` workspaces to a LArray Array object, use the [from_array()](../_generated/iode.Variables.from_array.rst#iode.Variables.from_array) and [to_array()](../_generated/iode.Variables.to_array.rst#iode.Variables.to_array) methods:

In [158]:
import larray as la

# ---- to LArray Array ----
# See Variables.to_array

arr_vars = variables.to_array()
print(f"Variables as LArray Array:\n{arr_vars.info}")

# ---- from LArray array ----
# See Variables.from_array

variables.from_array(arr_vars)

Variables as LArray Array:
400 x 56
 names [400]: 'ACAF' 'ACAG' 'AOUC' ... '_YOBS1' '_YRES0' '_YRES1'
 time [56]: '1960Y1' '1961Y1' '1962Y1' ... '2013Y1' '2014Y1' '2015Y1'
dtype: float64
memory used: 175.00 Kb


## Execute IODE report commands/files

To run an IODE command, use the [execute_command()](../_generated/iode.execute_command.rst#iode.execute_command) function:

In [159]:
from iode import execute_command

# ---- execute IODE commands ----
print("Execute IODE commands (useful for IODE functions and commands not yet ported to Python):")
execute_command("$WsClearVar")
execute_command("$WsSample 2000Y1 2005Y1")
execute_command("$DataCalcVar A t+1")
execute_command("$DataCalcVar B t-1")
execute_command("$DataCalcVar C A/B")
execute_command("$DataCalcVar D grt A")
execute_command("$WsSaveVar test_var.av")
with open("test_var.av", "r") as f:
    print(f.read())

Execute IODE commands (useful for IODE functions and commands not yet ported to Python):
Saving test_var.av
sample 2000Y1 2005Y1
A 1 2 3 4 5 6 
B -1 0 1 2 3 4 
C -1 na 3 2 1.66666666666667 1.5 
D na 100 50 33.3333333333333 25 20 



To run an entire IODE report (i.e. file with the '.rep' extension), call the [execute_report()](../_generated/iode.execute_report.rst#iode.execute_report) function:

In [160]:
from iode import execute_report

# ---- execute IODE reports ----
print("Execute an IODE report -> execute_report()")
with open("create_var.rep", "w") as f:
    f.write("$WsClearVar\n")
    f.write("$WsSample 2000Y1 2005Y1\n")
    f.write("$DataCalcVar %1% t+1 \n")
    f.write("$DataCalcVar %2% t-1 \n")
    f.write("$DataCalcVar %3% %1%/%2%\n")
    f.write("$DataCalcVar %4% grt %1% \n")
    f.write("$WsSaveVar test_var.av\n")

execute_report("create_var.rep", ["A", "B", "C", "D"])

with open("test_var.av", "r") as f:
    print(f.read())


Execute an IODE report -> execute_report()
Saving test_var.av
sample 2000Y1 2005Y1
A 1 2 3 4 5 6 
B -1 0 1 2 3 4 
C -1 na 3 2 1.66666666666667 1.5 
D na 100 50 33.3333333333333 25 20 

